Notebook del test

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from natsort import natsorted, natsort_keygen
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision.models.resnet import model_urls
import torchvision

In [2]:
data_path= os.path.join(r'C:\Users\Federico Cocchi\UNIMORE\3° SEMESTRE\covid-hackathon\TestSet', r'testClinData.xls')

In [3]:
df_test= pd.read_excel(data_path)
print(df_test.shape)
df_test.head(5)

(486, 40)


,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,...,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death
0,1247.0,P_3_29.png,F,76,1,1,36.0,4.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,1199.0,P_3_415.png,F,64,0,1,36.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,1504.0,P_3_163.png,F,87,1,1,37.6,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,1271.0,P_3_348.png,F,76,1,1,36.0,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,1572.0,P_3_263.png,F,77,1,1,36.1,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [4]:
df_test_order= df_test.sort_values(by=['ImageFile'], key= natsort_keygen())
df_test_order.reset_index(inplace=True, drop=True)
df_test_order.head(5)

,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,...,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death
0,1287.0,P_3_1.png,F,54,0,1,36.4,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1118.0,P_3_2.png,F,48,0,1,NaN,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,1342.0,P_3_3.png,F,88,1,1,36.5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,1552.0,P_3_4.png,F,67,0,1,38.9,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,1281.0,P_3_5.png,F,55,0,1,37.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


ordinato i dati seguendo il nome dell'immagine

In [5]:
df_test_order.to_excel('Test_order.xlsx')

In [6]:
transf_loader = A.Compose([
    A.Resize(224, 224),
    ToTensorV2(),
    # transforms.Normalize([0.5], [0.5])
])

def process_im(img, th=150):
    if type(img[0, 0]) != np.uint8:
        img = np.array((img * 255 / np.max(img)), dtype=np.uint8)

    if np.mean(img) <= th:
        img = img * -1 + 255

    return img

class custom_dataset_test(Dataset):
    def __init__(self, root_dir, transform=transf_loader, excel_file=None):
        self.dataframe = pd.read_excel(excel_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return self.dataframe.shape[0]

    def __getitem__(self, index):
        name_image = self.dataframe['ImageFile'][index]
        img_name = os.path.join(self.root_dir, name_image)

        img = np.array(Image.open(img_name))  # data augmentation
        img = process_im(img)
        img = self.transform(image=img)['image']
        return img, name_image

In [7]:
dataset_test = custom_dataset_test(excel_file=r'Test_order.xlsx',
                         root_dir=r'C:\Users\Federico Cocchi\UNIMORE\3° SEMESTRE\covid-hackathon\TestSet\TestSet',
                         transform=transf_loader)
print(len(dataset_test))

486


In [8]:
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

Load the model

In [9]:
model_urls['resnet101'] = model_urls['resnet101'].replace('https://', 'http://')
model = torchvision.models.resnet101(pretrained=True)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3),
                              bias=False)  # 1 channel in input image
num = model.fc.in_features  # 512
model.fc = torch.nn.Linear(num, 1)

print(num)
# summary(model, (1,224,224))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

2048


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Load the weights

In [10]:
model.load_state_dict(torch.load('model.pth', map_location= device))

<All keys matched successfully>

Test

In [11]:
correct = 0
total = 0
y_true= []
y_pred= []
image_names= []

with torch.no_grad():
    model.eval()
    for data in test_loader:
        images, names = data
        outputs = model(images.float())

        # provato ad applicare anche il softmax --> stesso risultato
        predicted_soft= torch.where(torch.sigmoid(outputs)>0.5, 1, 0)
        predicted = torch.where(outputs>0.5, 1, 0)

        total += predicted.squeeze().size(0)

        for i in range(predicted.squeeze().size(0)):
            y_pred.append(predicted.squeeze()[i].item())
            image_names.append(names[i])

In [12]:
df_test_mom = pd.DataFrame(list(zip(image_names, y_pred)), columns=['image_name', 'pred_class'])
df_test_mom.head(5)

,image_name,pred_class
0,P_3_1.png,1
1,P_3_2.png,1
2,P_3_3.png,1
3,P_3_4.png,0
4,P_3_5.png,0


In [14]:
df_test_mom.shape

(486, 2)

In [16]:
df_test_order.head(5)

,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,...,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death,pred_class
0,1287.0,P_3_1.png,F,54,0,1,36.4,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1118.0,P_3_2.png,F,48,0,1,NaN,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1
2,1342.0,P_3_3.png,F,88,1,1,36.5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1
3,1552.0,P_3_4.png,F,67,0,1,38.9,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
4,1281.0,P_3_5.png,F,55,0,1,37.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0


In [15]:
df_test_order['pred_class']= df_test_mom.pred_class

In [17]:
df_test_order.to_excel('Test_order_imagEval.xlsx')

## Load the csv

In [39]:
df_test_order= pd.read_excel(r'Test_order_imagEval.xlsx', index_col=0)
print(df_test_order.shape)
df_test_order.head(5)

(486, 41)


,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,...,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death,pred_class
0,1287.0,P_3_1.png,F,54,0,1,36.4,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1118.0,P_3_2.png,F,48,0,1,NaN,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1
2,1342.0,P_3_3.png,F,88,1,1,36.5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1
3,1552.0,P_3_4.png,F,67,0,1,38.9,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
4,1281.0,P_3_5.png,F,55,0,1,37.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0


In [40]:
df_train_order= pd.read_excel(r'Train_order_imagEval.xlsx', index_col=0)
print(df_train_order.shape)
df_train_order.head(5)

(1103, 41)


,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,...,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death,pred_class
0,1,P_1.png,A,38.0,0,1.0,36.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,MILD,0,0
1,900,P_1_1.png,F,96.0,0,1.0,36.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,NaN,1.0,MILD,0,0
2,901,P_1_2.png,F,62.0,1,1.0,NaN,4.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,NaN,1.0,MILD,0,0
3,902,P_1_3.png,F,57.0,1,1.0,39.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,NaN,1.0,SEVERE,0,1
4,903,P_1_4.png,F,41.0,1,1.0,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,1.0,SEVERE,0,1


In [41]:
ser_img_name= df_test_order.ImageFile

### continue wit the data

In [42]:
le = preprocessing.LabelEncoder()
y_train=le.fit_transform(df_train_order.Prognosis)
le.classes_

array(['MILD', 'SEVERE'], dtype=object)

In [43]:
y_train.shape

(1103,)

In [44]:
df_train_order.drop(['Row_number', 'Death', 'Prognosis', 'ImageFile', 'Hospital'], axis=1 , inplace=True)

In [45]:
mean_value= []
for i in range(df_train_order.shape[1]):
    mean= df_train_order.iloc[:,i].mean() # HERE
    mean_value.append(mean)

values= {}

for counter, value in enumerate(df_train_order.keys()):
    values[value] = mean_value[counter]
print(values)

{'Age': 64.60143328786812, 'Sex': 0.34632819582955576, 'PositivityAtAdmission': 0.9577067669172933, 'Temp_C': 37.55185577942736, 'DaysFever': 2.7229336437718277, 'Cough': 0.5227686703096539, 'DifficultyInBreathing': 0.5050045495905369, 'WBC': 7.100423848363599, 'RBC': 4.5722008041627245, 'CRP': 33.48366383380547, 'Fibrinogen': 602.5183823529412, 'Glucose': 124.8172757475083, 'PCT': 55.79216517857144, 'LDH': 362.63046153846153, 'INR': 1.3737686998504013, 'D_dimer': 2532.160158227848, 'Ox_percentage': 92.63477672530448, 'PaO2': 72.32052821128451, 'SaO2': 92.44488372093024, 'PaCO2': 33.402253694581276, 'pH': 7.457315721649485, 'CardiovascularDisease': 0.29979253112033194, 'IschemicHeartDisease': 0.13872135102533173, 'AtrialFibrillation': 0.08220603537981269, 'HeartFailure': 0.020833333333333332, 'Ictus': 0.035416666666666666, 'HighBloodPressure': 0.458592132505176, 'Diabetes': 0.15527950310559005, 'Dementia': 0.04569055036344756, 'BPCO': 0.06935817805383022, 'Cancer': 0.07349896480331262,

In [46]:
df_train_order= df_train_order.fillna(value=values)

In [47]:
df_train_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103 entries, 0 to 1102
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    1103 non-null   float64
 1   Sex                    1103 non-null   int64  
 2   PositivityAtAdmission  1103 non-null   float64
 3   Temp_C                 1103 non-null   float64
 4   DaysFever              1103 non-null   float64
 5   Cough                  1103 non-null   float64
 6   DifficultyInBreathing  1103 non-null   float64
 7   WBC                    1103 non-null   float64
 8   RBC                    1103 non-null   float64
 9   CRP                    1103 non-null   float64
 10  Fibrinogen             1103 non-null   float64
 11  Glucose                1103 non-null   float64
 12  PCT                    1103 non-null   float64
 13  LDH                    1103 non-null   float64
 14  INR                    1103 non-null   float64
 15  D_di

In [48]:
X_train= df_train_order
X_train.shape

(1103, 36)

Part with TEST data

In [34]:
df_test_order.head(5)

,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,WBC,RBC,CRP,...,HighBloodPressure,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,pred_class
0,54,0,1,36.400000,36.269643,0.0,1.0,6.40,4.96,10.46,...,134.0524,50.189231,32.093929,18.545833,38.924,750.743462,70.142593,21.057826,46.50037,1
1,48,0,1,49.028846,3.000000,0.0,0.0,3.67,4.97,1.70,...,0.0000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,46.50037,1
2,88,1,1,36.500000,1.000000,0.0,0.0,3.60,3.38,2.09,...,0.0000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,46.50037,1
3,67,0,1,38.900000,2.000000,0.0,0.0,3.20,5.09,7.36,...,0.0000,0.000000,1.000000,0.000000,0.000,0.000000,0.000000,0.000000,46.50037,0
4,55,0,1,37.000000,2.000000,0.0,0.0,4.78,4.87,1.65,...,0.0000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,46.50037,0


In [50]:
df_test_order.drop(['Row_number', 'Death', 'Prognosis', 'ImageFile', 'Hospital'], axis=1 , inplace=True)

In [51]:
mean_value= []
for i in range(df_test_order.shape[1]):
    mean= df_test_order.iloc[:,i].mean()
    mean_value.append(mean)

values= {}

for counter, value in enumerate(df_test_order.keys()):
    values[value] = mean_value[counter]
print(values)

{'Age': 68.00617283950618, 'Sex': 0.4279835390946502, 'PositivityAtAdmission': 1.0, 'Temp_C': 36.8875, 'DaysFever': 2.315625, 'Cough': 0.2768595041322314, 'DifficultyInBreathing': 0.5578512396694215, 'WBC': 6.6595, 'RBC': 4.591604166666667, 'CRP': 8.151085106382979, 'Fibrinogen': 519.2857142857143, 'Glucose': 127.77870563674321, 'PCT': 0.3413118279569893, 'LDH': 321.6354679802956, 'INR': 1.0844588744588743, 'D_dimer': 1637.3026905829597, 'Ox_percentage': nan, 'PaO2': 74.1055900621118, 'SaO2': 93.36943127962085, 'PaCO2': 33.22, 'pH': 7.445869565217391, 'CardiovascularDisease': nan, 'IschemicHeartDisease': nan, 'AtrialFibrillation': nan, 'HeartFailure': nan, 'Ictus': nan, 'HighBloodPressure': 0.5104166666666666, 'Diabetes': 0.14791666666666667, 'Dementia': 0.04791666666666667, 'BPCO': 0.06041666666666667, 'Cancer': 0.06041666666666667, 'ChronicKidneyDisease': 0.04375, 'RespiratoryFailure': 0.01875, 'Obesity': 0.0625, 'Position': nan, 'pred_class': 0.5370370370370371}


In [52]:
df_test_order= df_test_order.fillna(value=values)

In [55]:
values = {"Ox_percentage": 0, "CardiovascularDisease": 0, "IschemicHeartDisease": 0, "AtrialFibrillation": 0,
          "HeartFailure": 0, "Ictus": 0, "Position": 0}
df_test_order.fillna(value=values, inplace=True)

In [57]:
df_test_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 0 to 485
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    486 non-null    int64  
 1   Sex                    486 non-null    int64  
 2   PositivityAtAdmission  486 non-null    int64  
 3   Temp_C                 486 non-null    float64
 4   DaysFever              486 non-null    float64
 5   Cough                  486 non-null    float64
 6   DifficultyInBreathing  486 non-null    float64
 7   WBC                    486 non-null    float64
 8   RBC                    486 non-null    float64
 9   CRP                    486 non-null    float64
 10  Fibrinogen             486 non-null    float64
 11  Glucose                486 non-null    float64
 12  PCT                    486 non-null    float64
 13  LDH                    486 non-null    float64
 14  INR                    486 non-null    float64
 15  D_dime

In [58]:
X_test= df_test_order
X_test.shape

(486, 36)

## Classification

In [63]:
# 1
dt= DecisionTreeClassifier(criterion='entropy', max_features=None, min_samples_split=5)
dt.fit(X_train, y_train)

pred_test_dt= dt.predict(X_test)
#print('test: ', accuracy_score(y_test, pred_test) )

In [59]:
# 2
randomforest= RandomForestClassifier(n_estimators=1000)
# criterion='entropy', max_features=10, min_samples_split=2, n_estimators=100
randomforest.fit(X_train, y_train)

pred_test_rf= randomforest.predict(X_test)
#print('test: ', accuracy_score(y_test, pred_test) )

Build the output

## 1

In [60]:
df_final = pd.DataFrame(list(zip(ser_img_name, pred_test_rf)), columns=['image_name', 'pred_class'])
df_final.head()

,image_name,pred_class
0,P_3_1.png,1
1,P_3_2.png,1
2,P_3_3.png,1
3,P_3_4.png,0
4,P_3_5.png,0


In [61]:
dict_rep= {0: 'MILD', 1: 'SEVERE'}
df_final.pred_class= df_final.pred_class.replace(dict_rep)
df_final.head()

,image_name,pred_class
0,P_3_1.png,SEVERE
1,P_3_2.png,SEVERE
2,P_3_3.png,SEVERE
3,P_3_4.png,MILD
4,P_3_5.png,MILD


In [62]:
df_final.to_csv('classification_rf_final.txt', header=False, index=False)

## 2

In [64]:
df_final = pd.DataFrame(list(zip(ser_img_name, pred_test_dt)), columns=['image_name', 'pred_class'])
df_final.head()

,image_name,pred_class
0,P_3_1.png,1
1,P_3_2.png,0
2,P_3_3.png,1
3,P_3_4.png,0
4,P_3_5.png,0


In [65]:
dict_rep= {0: 'MILD', 1: 'SEVERE'}
df_final.pred_class= df_final.pred_class.replace(dict_rep)
df_final.head()

,image_name,pred_class
0,P_3_1.png,SEVERE
1,P_3_2.png,MILD
2,P_3_3.png,SEVERE
3,P_3_4.png,MILD
4,P_3_5.png,MILD


In [66]:
df_final.to_csv('classification_dt_final.txt', header=False, index=False)